# Initial Deceleration Rate (DR) Extraction with OSM + OVTL Intersection Control

## 1. Project Context

**Goal:** Compute initial deceleration rate (DR) for vehicles during intersection conflict windows, and attach:
- Intersection control type (`signal`, `stop`, `unknown`) using **OSM baseline + OVTL upgrades**
- Traffic light signal state (from vehicle-observed `TrafficLights.Signal`, when available)
- Units (m/s² or ft/s²)

In this implementation, we detect the initial braking moment using brake flags and/or negative acceleration thresholds. If no braking event is detected within a conflict window, the window is currently skipped and no DR is recorded in the output CSV.

**Data sources:**
- MotionData via HistoryData API (trajectory + brake + traffic light observations)
- Intersection lane geometry (GeoJSON)
- OSM (traffic signals / stop signs)

DR is used as a surrogate safety measure: it captures how abruptly a vehicle decelerates at the onset of braking,
or (if no clear braking occurs) the lowest acceleration observed within the conflict window.


## 2. Definition: Initial Deceleration Rate (DR)

**Initial deceleration rate (DR)** is defined as the instantaneous deceleration of the subject vehicle at the moment braking begins.

Operational definition used in this pipeline:
- **If braking is detected:** DR = the **first negative acceleration** at braking onset (or first brake-flagged sample with valid acceleration).
- **If braking is not detected:** DR = the **minimum (most negative) acceleration** observed during the conflict window.

When acceleration is not directly available, acceleration is estimated from speed using finite differences.


## 3. References

- Gettman & Head (2003). *Surrogate Safety Measures from Traffic Simulation Models*, FHWA-RD-03-050.
  PDF: https://ntlrepository.blob.core.windows.net/lib/38000/38000/38015/FHWA-RD-03-050.pdf

- Gettman et al. (2008). *Surrogate Safety Assessment Model and Validation: Final Report*, FHWA-HRT-08-051.
  PDF: https://www.fhwa.dot.gov/publications/research/safety/08051/08051.pdf


## 4. DR Logic Diagram

**A. Data retrieval**
- Fetch MotionData in time slices (stability + API caps)

**B. Intersection control labeling**
- Load intersection lane geometry (GeoJSON)
- Label intersection control using OSM (baseline: stop/signals)
- Build OVTL evidence points from MotionData (vehicle-observed traffic lights)
- Upgrade control labels using OVTL (unknown → signal when evidence is strong)

**C. Trajectory + conflict analysis**
- Build per-vehicle trajectories from MotionData (speed, accel, brake flag, traffic light signal)
- Annotate each trajectory point with distance-to-intersection + control type
- Extract conflict windows (contiguous segments within distance threshold)
- Compute DR per conflict window (brake onset vs min-acc fallback)
- Attach units + output CSV


## 5. Imports and Config

Key constants:
- API endpoint + brake keys
- vehicle mapping (`chid_to_vehicle`)
- thresholds used in OVTL filtering (distance), and DR fallback threshold
- the CSV output schema used for downstream analysis

In [18]:
import json, math, time, random
import requests
import pandas as pd
import numpy as np

import geopandas as gpd
from shapely.geometry import Point, LineString
from pyproj import Transformer
from zoneinfo import ZoneInfo
from datetime import datetime, timedelta, timezone
from collections import defaultdict, Counter
import csv
import osmnx as ox

In [13]:
getHistoryDataAPI = "http://70.229.15.100:9080/txvapi/history/getHistoryData"
BRAKE_KEY = "Brake"
BRAKE_VALID_KEY = "isBrakeCmdActive"
NEG_ACCEL_FALLBACK = -0.3
# Valid SignalType values that correspond to real signals
# 1 = DEFAULT_STANDARD, 2 = 5_SEGMENT, 4 = VALID_UNKNOWN
REAL_SIGNAL_TYPES = {1, 2, 4}

MAX_OVTL_DISTANCE_M = 120.0

chid_to_vehicle = {
    19200: "mallory",
    19201: "megalodon",
    19202: "morizo",
    19203: "mav",
    19204: "metatron",
    19205: "marymae",
    19206: "mastermind",
}

csv_columns = [
    "chid",
    "vehicle_name",
    "start_time",
    "end_time",
    "brake_time",
    "initial_DR_mps2",
    "lat",
    "lon",
    "distance_to_intersection_m",
    "intersection_node_id",
    "intersection_control",
    "ovtl_distance_at_brake",
    "ovtl_flag_at_brake",
    "ovtl_signal_at_brake",
    "lane_match",
]

INPUT_JSON = "motiondata_sample.json"

## 6. Baseline intersection control labeling using OSM

We label intersection lane segments as:
- `signal` if near OSM `highway=traffic_signals`
- `stop` if near OSM `highway=stop`
- otherwise `unknown`

In [15]:
def fetch_osm_control_points_from_lanes(lane_gdf):
    """
    Use OSMnx + Overpass to download all highway=traffic_signals and
    highway=stop nodes inside convex hull of lane_gdf.
    Returns a GeoDataFrame in the SAME CRS as lane_gdf.
    """
    lane_wgs = lane_gdf.to_crs(epsg=4326)
    base_poly = lane_wgs.unary_union.convex_hull

    # Buffer ~100m in degrees (~0.001 deg ≈ 100m)
    area_poly = base_poly.buffer(0.001)

    tags = {"highway": ["traffic_signals", "stop"]}
    osm_features = ox.features_from_polygon(area_poly, tags=tags)
    osm_features = osm_features.to_crs(lane_gdf.crs)

    osm_points = osm_features[osm_features.geometry.type == "Point"].copy()

    # DEBUG
    if not osm_points.empty:
        print("OSM control points:", len(osm_points))
        print(osm_points["highway"].value_counts())
    else:
        print("No OSM control points found in area.")

    return osm_points


def annotate_intersections_with_osm(intersection_gdf, osm_points_gdf,
                                    signal_radius=80.0, stop_radius=80.0):
    """
    Given intersection_gdf (with geometry in UTM, here intersection *lanes*)
    and OSM points with highway=traffic_signals / highway=stop in same CRS,
    classify each intersection lane as 'signal', 'stop', or leave as 'unknown'.
    """
    inter = intersection_gdf.copy()

    if "control" not in inter.columns:
        inter["control"] = "unknown"

    if osm_points_gdf.empty:
        return inter

    signals = osm_points_gdf[osm_points_gdf["highway"] == "traffic_signals"].copy()
    stops   = osm_points_gdf[osm_points_gdf["highway"] == "stop"].copy()

    # Build spatial indexes (if non-empty)
    signal_sindex = signals.sindex if not signals.empty else None
    stop_sindex   = stops.sindex if not stops.empty else None

    new_controls = []

    for idx, row in inter.iterrows():
        geom = row.geometry
        current_ctrl = row.get("control", "unknown")
        ctrl = current_ctrl

        # Use the centroid of the intersection lane for control lookup
        if geom is None or geom.is_empty:
            new_controls.append(ctrl)
            continue

        pt = geom.centroid

        # 1) Nearby signal?
        is_signal = False
        if signal_sindex is not None:
            bbox = pt.buffer(signal_radius).bounds
            candidate_idx = list(signal_sindex.intersection(bbox))
            if candidate_idx:
                cand = signals.iloc[candidate_idx]
                dists = cand.distance(pt)
                if not dists.empty and float(dists.min()) <= signal_radius:
                    ctrl = "signal"
                    is_signal = True

        # 2) If no signal, nearby stop?
        if (not is_signal) and (stop_sindex is not None):
            bbox = pt.buffer(stop_radius).bounds
            candidate_idx = list(stop_sindex.intersection(bbox))
            if candidate_idx:
                cand = stops.iloc[candidate_idx]
                dists = cand.distance(pt)
                if not dists.empty and float(dists.min()) <= stop_radius:
                    if ctrl != "signal":
                        ctrl = "stop"

        new_controls.append(ctrl)

    inter["control"] = new_controls
    return inter

## 7. OVTL upgrades: using repeated vehicle-observed evidence to refine control labels

OSM can miss traffic signal nodes or be incomplete in certain regions.

We therefore upgrade `unknown` to `signal` using OVTL evidence:
- OVTL points are derived from MotionData `TrafficLights`
- clustered into repeated observation points with hit counts
- if enough hits occur near an intersection lane centroid, we upgrade its control type

Importantly: this logic only upgrades **unknown** intersections and does not overwrite `stop` or existing `signal`.


In [16]:
def annotate_intersections_with_ovtl(intersection_gdf, ovtl_points_gdf,
                                     ovtl_radius=8.0,
                                     min_hits=5):
    """
    - Uses small radius (8m) around lane centroid
    - Requires at least min_hits OVTL events (clustered, counted)
    - Only upgrades 'unknown' to 'signal'
    - Never overwrites 'stop' or existing 'signal'
    """
    inter = intersection_gdf.copy()

    if ovtl_points_gdf is None or ovtl_points_gdf.empty:
        print("No OVTL points provided; skipping OVTL-based annotation.")
        return inter

    sindex = ovtl_points_gdf.sindex
    new_controls = []
    upgrades = 0

    for idx, row in inter.iterrows():
        geom = row.geometry
        ctrl = row.get("control", "unknown")

        # Only consider upgrading unknown
        if ctrl != "unknown":
            new_controls.append(ctrl)
            continue

        if geom is None or geom.is_empty:
            new_controls.append(ctrl)
            continue

        pt = geom.centroid

        # Search OVTL clusters within ovtl_radius
        bbox = pt.buffer(ovtl_radius).bounds
        cand_idx = list(sindex.intersection(bbox))
        if not cand_idx:
            new_controls.append(ctrl)
            continue

        cand = ovtl_points_gdf.iloc[cand_idx]
        dists = cand.geometry.distance(pt)
        if dists.empty:
            new_controls.append(ctrl)
            continue

        near_mask = dists <= ovtl_radius
        cand_near = cand[near_mask]

        if cand_near.empty:
            new_controls.append(ctrl)
            continue

        # Use total OVTL hit count near this intersection
        total_hits = cand_near["count"].sum()

        if total_hits >= min_hits:
            ctrl = "signal"
            upgrades += 1

        new_controls.append(ctrl)

    inter["control"] = new_controls
    print(f"OVTL upgrades applied (unknown → signal): {upgrades}")
    return inter


### Step 0 - Load lane geometry and extract intersection lane segments

The pipeline uses lane geometry to compute distance-to-intersection.
We first load the GeoJSON lanes, filter down to intersection-labeled lane segments,
and initialize `control="unknown"` before applying OSM and OVTL labeling.

The resulting `intersection_df` is the geometry reference used for:
- distance-to-nearest intersection computation
- conflict window detection
- attaching intersection control labels to trajectory points

In [19]:
def remove_z(geom):
    if isinstance(geom, LineString) and geom.has_z:
        return LineString([(x, y) for x, y, z in geom.coords])
    return geom


gis_lane_df = gpd.read_file("ca-martinez-carquinez_Oct8th2024.geojson")
gis_lane_df = gis_lane_df.loc[gis_lane_df["type_names"] == "Lane Nominal"].copy()
gis_lane_df["geometry"] = gis_lane_df["geometry"].apply(remove_z)
gis_lane_df = gis_lane_df.to_crs(epsg=26910)

# Use ONLY lane segments whose semantic_description == "intersection"
intersection_df = gis_lane_df[
    gis_lane_df["semantic_description"].str.lower() == "intersection"
].copy()
intersection_df = intersection_df.reset_index(drop=True)
intersection_df["control"] = "unknown"

print("Number of intersection lanes:", len(intersection_df))

# Fetch OSM controls and annotate these intersection lanes
osm_controls = fetch_osm_control_points_from_lanes(gis_lane_df)
intersection_df = annotate_intersections_with_osm(
    intersection_df, osm_controls, signal_radius=60.0, stop_radius=60.0
)

print("Intersection control counts (OSM only):")
print(intersection_df["control"].value_counts())

wgs84_to_utm = Transformer.from_crs("EPSG:4326", gis_lane_df.crs, always_xy=True)


# Lane matching helper
# ---------------------------------------------------------------------

def is_point_on_linestring(point, linestring, error_bound):
    buffered_linestring = linestring.buffer(error_bound)
    return buffered_linestring.intersects(point)


def find_lane(point, initial_errMargin=1.0, max_errMargin=30.0, step=1,
              allowed_semantics=None):
    current_margin = initial_errMargin

    if allowed_semantics is not None:
        df = gis_lane_df[gis_lane_df["semantic_description"].isin(allowed_semantics)].copy()
    else:
        df = gis_lane_df.copy()

    while current_margin <= max_errMargin:
        matched_lanes = []
        distances = df["geometry"].distance(point)
        df["dist_to_point"] = distances

        for row_index in df.index:
            if df.loc[row_index, "dist_to_point"] > current_margin:
                continue
            if is_point_on_linestring(point, df.loc[row_index, "geometry"], current_margin):
                matched_lanes.append({
                    "Lane id": df.loc[row_index, "id"],
                    "Source id": df.loc[row_index, "target_id"],
                    "Target id": df.loc[row_index, "source_id"],
                    "Semantic desc": df.loc[row_index, "semantic_description"],
                })

        if len(matched_lanes) == 1:
            return matched_lanes
        elif len(matched_lanes) > 1:
            return [matched_lanes[0]]

        current_margin += step

    return []

Skipping field right_boundary_ids: unsupported OGR type: 5
Skipping field left_boundary_ids: unsupported OGR type: 5


Number of intersection lanes: 2448


/var/folders/h6/qd2kpqfj4_35f63hmcw_m0nc0000gn/T/ipykernel_61771/2867093252.py:8: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  base_poly = lane_wgs.unary_union.convex_hull


OSM control points: 183
highway
stop               163
traffic_signals     20
Name: count, dtype: int64
Intersection control counts (OSM only):
control
stop       1199
unknown    1094
signal      155
Name: count, dtype: int64


### Step 1 — Fetch Motion Data (Time-Sliced)

The function `fetch_data(...)` retrieves motion data for a single continuous
time window specified by a start and end timestamp.

Because large time ranges can exceed API response limits, the full retrieval
process uses **time slicing**:
- the overall analysis window is divided into smaller intervals
- `fetch_data` is called once per slice
- results are concatenated into a single dataset


In [6]:
def fetch_data(data_type, start_time, end_time):
    params = {
        "DataType": data_type,
        "FilterFlags": 2,
        "TimeFilter": {
            "StartTime": start_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "EndTime": end_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        },
    }

    response = requests.post(getHistoryDataAPI, json=params)
    response.raise_for_status()
    return json.loads(response.text)

def fetch_motiondata_sliced(start_time, end_time, chunk_minutes=10):
    """
    Fetch MotionData from API in chunks and return {"Response": [...]}.
    """
    all_resp = []
    slice_start = start_time

    while slice_start < end_time:
        slice_end = min(slice_start + timedelta(minutes=chunk_minutes), end_time)

        print(f"\nFetching MotionData from {slice_start} to {slice_end} ...")
        chunk = fetch_data("MotionData", slice_start, slice_end)
        chunk_resp = chunk.get("Response", [])
        print(f"  → got {len(chunk_resp)} records")

        if len(chunk_resp) >= 200000:
            print("  ⚠ WARNING: hit 200000-cap in this slice. Consider smaller chunk_minutes.")

        all_resp.extend(chunk_resp)
        slice_start = slice_end

    print(f"\nTotal MotionData records across all slices: {len(all_resp)}")
    return {"Response": all_resp}


def generate_motiondata_json(start_str, end_str,
                             chunk_minutes=10,
                             output_json="motiondata_sample.json"):
    """
    Convenience function to hit API and dump MotionData to JSON.
    """
    start_time = datetime.strptime(start_str, "%Y-%m-%d %H:%M:%S")
    end_time   = datetime.strptime(end_str,   "%Y-%m-%d %H:%M:%S")

    data = fetch_motiondata_sliced(start_time, end_time, chunk_minutes=chunk_minutes)

    with open(output_json, "w") as f:
        json.dump(data, f)

    print(f"Saved MotionData JSON to: {output_json}")

### Step 2 — MotionData parsing helpers: speed and acceleration standardization

MotionData may contain:
- direct `Speed`, or velocity vectors under `TELE_OP_OBJECTS["vel"]`
- acceleration vectors under `POSE.Acc`

This notebook uses:
- `scalar_speed_from_vel(...)` to compute speed if Speed is missing
- `parse_scalar_acceleration(...)` to compute signed scalar acceleration from POSE.Acc


In [20]:
def parse_scalar_acceleration(content):
    """
    May Mobility's required scalar acceleration:
    - Use POSE.Acc = [ax, ay, az]
    - magnitude = sqrt(ax^2 + ay^2)
    - sign from ax
    - round to 1 decimal
    """
    pose = content.get("POSE") or {}
    acc_vec = pose.get("Acc")
    if not isinstance(acc_vec, (list, tuple)) or len(acc_vec) < 2:
        return None

    try:
        a_x = float(acc_vec[0])
        a_y = float(acc_vec[1])
    except (TypeError, ValueError):
        return None

    magnitude = math.sqrt(a_x * a_x + a_y * a_y)
    if a_x < 0:
        magnitude = -magnitude

    # rounding according to May's example
    return round(magnitude, 1)


def scalar_speed_from_vel(vel_vec):
    """
    vel_vec: list/tuple [v_x, v_y, v_z] from TELE_OP_OBJECTS["vel"]
    Returns scalar speed in m/s.
    """
    if vel_vec is None or len(vel_vec) < 3:
        return None

    try:
        v_x = float(vel_vec[0])
        v_y = float(vel_vec[1])
        v_z = float(vel_vec[2])
    except (TypeError, ValueError):
        return None

    return math.sqrt(v_x**2 + v_y**2 + v_z**2)

### Step 3 — Conflict window logic: distance-to-intersection thresholding

For each trajectory point, we compute:
- distance to the nearest intersection lane segment
- `intersection_control` from the nearest intersection lane

Conflict windows are contiguous sequences where:
- distance_to_intersection <= threshold

These windows are the temporal regions where we compute DR.

In [4]:
def distance_to_nearest_intersection(point_utm):
    """
    Distance from this point to the nearest *intersection lane segment*.
    Returns (distance_in_meters, control_label) or (None, None).
    control_label is one of: "signal", "stop", "unknown".
    """
    if intersection_df.empty:
        return None, None

    dists = intersection_df["geometry"].distance(point_utm)
    if dists.empty:
        return None, None

    idx_min = dists.idxmin()
    row = intersection_df.loc[idx_min]
    dist = float(dists.loc[idx_min])
    ctrl = row.get("control", "unknown")

    return dist, ctrl
    
def annotate_traj_with_intersection_distance(traj):
    for p in traj:
        lat = p.get("lat")
        lon = p.get("lon")
        if lat is None or lon is None:
            p["dist_to_intersection_m"] = None
            p["nearest_intersection_node"] = None
            p["intersection_control"] = None
            continue

        x, y = wgs84_to_utm.transform(lon, lat)  # (lon, lat)
        pt_utm = Point(x, y)

        dist, ctrl = distance_to_nearest_intersection(pt_utm)
        p["dist_to_intersection_m"] = dist
        p["nearest_intersection_node"] = None
        p["intersection_control"] = ctrl

def get_conflict_windows_for_traj(traj, dist_threshold_m=30.0, min_points=3):
    windows = []
    in_window = False
    start_i = None
    n = len(traj)

    for i, p in enumerate(traj):
        d = p.get("dist_to_intersection_m")
        in_conflict = (d is not None and d <= dist_threshold_m)

        if in_conflict and not in_window:
            in_window = True
            start_i = i
        elif not in_conflict and in_window:
            if i - start_i >= min_points:
                windows.append((start_i, i - 1))
            in_window = False

    if in_window and n - start_i >= min_points:
        windows.append((start_i, n - 1))

    return windows

### Step 4 — Build per-vehicle trajectories from MotionData

We build time-sorted trajectories per `chid`. Each point includes:
- time, speed, lat/lon
- brake flag (derived from Brake + isBrakeCmdActive)
- scalar acceleration (from POSE.Acc)
- `traffic_light_signal` and `traffic_light_utime` from `TrafficLights[0]` if present


In [21]:
def build_trajectories_from_response(response_json):
    trajectories = defaultdict(list)
    resp = response_json.get("Response", [])

    n_points = 0
    n_nonempty_tl = 0

    for obj in resp:
        chid = obj.get("Chid")
        if chid is None:
            continue

        json_str = obj.get("Json")
        if not json_str:
            continue

        try:
            content = json.loads(json_str)
        except json.JSONDecodeError:
            continue

        time_str = obj.get("Time")
        if not time_str:
            continue

        t = None
        for fmt in ("%Y-%m-%dT%H:%M:%S.%f", "%Y-%m-%dT%H:%M:%S"):
            try:
                t = datetime.strptime(time_str, fmt)
                break
            except ValueError:
                continue
        if t is None:
            continue

        speed = content.get("Speed")
        if speed is None:
            tele = content.get("TELE_OP_OBJECTS") or {}
            vel = tele.get("vel")
            speed = scalar_speed_from_vel(vel)

        if speed is None:
            continue

        lat = content.get("Latitude")
        lon = content.get("Longitude")
        lat = float(lat) if lat is not None else None
        lon = float(lon) if lon is not None else None

        brake_val = content.get(BRAKE_KEY, None)
        brake_active = content.get(BRAKE_VALID_KEY, None)

        if not brake_active or brake_val is None:
            brake_flag = False
            brake_value = None
        else:
            try:
                brake_value = float(brake_val)
            except (TypeError, ValueError):
                brake_value = None

            if brake_value is None:
                brake_flag = False
            else:
                brake_flag = abs(brake_value) > 1e-6

        acc_scalar = parse_scalar_acceleration(content)

        traffic_lights = content.get("TrafficLights", None)
        tl_signal = None
        tl_utime = None

        if isinstance(traffic_lights, list) and len(traffic_lights) > 0:
            tl0 = traffic_lights[0]
            if isinstance(tl0, dict):
                tl_signal = tl0.get("Signal")
                tl_utime = tl0.get("UTime")
                n_nonempty_tl += 1

        trajectories[chid].append(
            {
                "time": t,
                "speed": speed,
                "lat": lat,
                "lon": lon,
                "brake_flag": brake_flag,
                "acc_scalar": acc_scalar,
                "brake_value": brake_value,
                "traffic_light_signal": tl_signal,
                "traffic_light_utime": tl_utime,
            }
        )

        n_points += 1

    for chid in trajectories:
        trajectories[chid].sort(key=lambda p: p["time"])

    print("\n=== OVTL / TrafficLights debug summary (Speed-filtered trajectories) ===")
    print("Total trajectory points:", n_points)
    print("Points with non-empty TrafficLights list:", n_nonempty_tl)
    print(f"Built trajectories for {len(trajectories)} chids: {list(trajectories.keys())[:10]}")
    return trajectories

### Step 5 — Compute DR in a Conflict Window

Within each conflict window:
1) If acceleration exists, prefer **first (brake_flag=True AND acc_scalar present)**
2) If not found, fallback to **first acc_scalar < NEG_ACCEL_FALLBACK**
3) If acceleration is missing, compute finite-difference acceleration from speed and apply the same threshold

When a braking point is found, we output:
- brake time
- DR value
- intersection control at brake moment
- traffic light signal at brake moment (TrafficLights.Signal)
- distance to intersection at brake time
- lane match information


In [10]:
def compute_initial_braking_event_in_window(traj, start_idx, end_idx,
                                            neg_thresh=NEG_ACCEL_FALLBACK):
    window = traj[start_idx:end_idx + 1]
    if len(window) < 3:
        return None, None, None, None, [], None, None, None, None, None

    times = [p["time"] for p in window]
    speeds = [p["speed"] for p in window]
    brake_flags = [p.get("brake_flag", False) for p in window]
    acc_scalars = [p.get("acc_scalar") for p in window]
    tl_signals  = [p.get("traffic_light_signal") for p in window]

    t0 = times[0]
    t_sec = [(t - t0).total_seconds() for t in times]

    braking_idx = None
    dr_mps2 = None

    has_acc = any(a is not None for a in acc_scalars)

    # 1) brake_flag + acc
    if has_acc:
        for i, flag in enumerate(brake_flags):
            if flag and acc_scalars[i] is not None:
                braking_idx = i
                dr_mps2 = acc_scalars[i]
                break

    # 2) fallback: just negative acc
    if braking_idx is None and has_acc:
        for i, a in enumerate(acc_scalars):
            if a is not None and a < neg_thresh:
                braking_idx = i
                dr_mps2 = a
                break

    # 3) fallback: finite difference on speed
    if braking_idx is None and not has_acc:
        accel_values = []
        n = len(speeds)
        for i in range(n):
            if i == 0:
                dv = speeds[i + 1] - speeds[i]
                dt = t_sec[i + 1] - t_sec[i]
            elif i == n - 1:
                dv = speeds[i] - speeds[i - 1]
                dt = t_sec[i] - t_sec[i - 1]
            else:
                dv = speeds[i + 1] - speeds[i - 1]
                dt = t_sec[i + 1] - t_sec[i - 1]

            accel_values.append(dv / dt if dt != 0 else 0.0)

        for i, a in enumerate(accel_values):
            if a < neg_thresh:
                braking_idx = i
                dr_mps2 = a
                break

    if braking_idx is None:
        return None, None, None, None, [], None, None, None, None, None

    brake_time = window[braking_idx]["time"]
    lat = window[braking_idx].get("lat")
    lon = window[braking_idx].get("lon")
    dist_to_intersection_m = window[braking_idx].get("dist_to_intersection_m")
    intersection_control = window[braking_idx].get("intersection_control")
    brake_node_id = window[braking_idx].get("nearest_intersection_node")
    tl_signal_at_brake = tl_signals[braking_idx]

    lane_match = []
    if lat is not None and lon is not None:
        brake_point_geo = gpd.GeoSeries(
            [Point(lon, lat)], crs="EPSG:4326"
        ).to_crs(gis_lane_df.crs).iloc[0]
        lane_match = find_lane(
            brake_point_geo, allowed_semantics=["street", "intersection"]
        )

    return (
        brake_time,
        dr_mps2,
        lat,
        lon,
        lane_match,
        dist_to_intersection_m,
        intersection_control,
        brake_node_id,
        None,                # ovtl_distance_at_brake (no Distance in API)
        tl_signal_at_brake,  
    )

### Step 6 - Run full pipeline

This final part of the python file runs the full pipeline:

1. Retrieve MotionData using time slicing
2. Build OVTL clusters from MotionData
3. Upgrade intersection control labels with OVTL (change unknown to signal)
4. Build trajectories
5. Annotate trajectories with distance-to-intersection and control
6. Extract conflict windows
7. Compute DR for each window and export to CSV

Note: In the current implementation, if no braking event is detected in a window,
that window is skipped (rather than returning min-acceleration DR).